# Predictions

For calculating the predictions regarding the i) number of infections, ii) number of hospitalisations, iii) number of Intensive Care Units and iv) number of deaths, we'll use ARMA - Autoregressive Moving Average 

In [1]:
from imports_functions import *
%matplotlib inline

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
db1 = pd.read_csv('covid-19-sample.csv')

In [3]:
db1 =clean_data(db1)

We'll treat the data as a time series. For that, we shall aplply evolution function and set the date as index:

In [4]:
bydate=evolution(db1).set_index('date')

## Infections

In [5]:
def add_days(dataframe):
    
    dataframe = dataframe.append(pd.Series(name='2021-06-11 00:00:00'))
    dataframe = dataframe.append(pd.Series(name='2021-06-12 00:00:00'))
    dataframe = dataframe.append(pd.Series(name='2021-06-13 00:00:00'))
    dataframe = dataframe.append(pd.Series(name='2021-06-14 00:00:00'))
    dataframe = dataframe.append(pd.Series(name='2021-06-15 00:00:00'))
    dataframe = dataframe.append(pd.Series(name='2021-06-16 00:00:00'))
    dataframe = dataframe.append(pd.Series(name='2021-06-17 00:00:00'))
    
    return dataframe

In [6]:
infections = training_models(bydate, 'infections')

In [8]:
def pred_infections(dataframe):

        model = ARMA(dataframe, order = (4, 2)).fit(disp=False)
        infections = add_days(dataframe)
        train = infections[:-7]
        return pd.DataFrame(model.predict(len(train), len(infections)-1), columns=['pred_infections'])

In [10]:
pred_infections = pred_infections(infections)

/home/ffreire/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  warnings.warn('No frequency information was'


In [11]:
pred_infections

,pred_infections
2021-06-11,3436.980030
2021-06-12,3383.025769
2021-06-13,3347.409818
2021-06-14,3330.748341
2021-06-15,3328.881073
2021-06-16,3341.760262
2021-06-17,3369.143182


In [ ]:
infections

In [ ]:
model_infections = ARMA(infections, order = (4, 2)).fit(disp=False)

In [ ]:
infections = add_days(infections)

In [ ]:
train = infections[:-7]

In [ ]:
train

In [ ]:
pred_infections = model.predict(len(train), len(infections)-1)

In [ ]:
pred_infections

In [ ]:
hosp = training_models(bydate, 'hospitalisations')
icu = training_models(bydate, 'icu')
dead = training_models(bydate, 'deaths')

We should train an ARMA model for a given p and q, for a given set of days. After several experiments and simulations, one found that we minimise the root square mean error if we train the model with the whole dataset, and predict on forward:

In [ ]:
model = ARMA(train, order = (4, 2)).fit(disp=False)
pred = model.predict(len(train), len(infections)-1)

In [ ]:
infections = add_days(infections)

In [ ]:
train = infections[:-7]

In [ ]:
pred_1 = pd.DataFrame(pred, columns = ['pred_1'])

In [ ]:
predictions = pd.read_csv('predictions.csv')

In [ ]:
predictions = predictions.rename(columns={'Unnamed: 0': 'date'})

In [ ]:
predictions

In [ ]:
pred_2 = predictions[['date', 'pred_num_infections']]

In [ ]:
pred_2 = pred_2.set_index('date')

In [ ]:
db2 = pd.read_csv('covid-19-sample_new.csv')

In [ ]:
db2 =clean_data(db2)

In [ ]:
bydate2 = evolution(db2).set_index('date')

In [ ]:
test = bydate2.tail(4)

In [ ]:
test = pd.DataFrame(test.ave_7_num_infections, columns = ['ave_7_num_infections'])

In [ ]:
test = pd.concat([test, pred_1], axis=1)

In [ ]:
test = pd.concat([test, pred_2], axis=1)

In [ ]:
test

In [ ]:
plt.rcParams['figure.figsize']=(15, 15)
plt.plot(test.ave_7_num_infections);
plt.plot(test.pred_1, 'r')
plt.plot(test.pred_num_infections, 'black')

In [ ]:
db3 = pd.read_csv('covid-19-sample_val.csv')
db3 = clean_data(db3)
bydate3 = evolution(db3).set_index('date')

In [ ]:
val = bydate3.tail(7)

In [ ]:
val = pd.DataFrame(val.ave_7_num_infections, columns = ['ave_7_num_infections'])
val = pd.concat([val, pred_1], axis = 1)
val = pd.concat([val, pred_2], axis = 1)

In [ ]:
plt.rcParams['figure.figsize']=(15, 15)
plt.plot(val.ave_7_num_infections);
plt.plot(val.pred_1, 'r')
plt.plot(val.pred_num_infections, 'black')